In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import numpy as np; np.random.seed(42)
import seaborn as sns

import pyLDAvis
import pyLDAvis.gensim
from collections import OrderedDict
from itertools import combinations
import MeCab
from konlpy.tag import *
import nltk
from nltk.corpus import stopwords
from gensim import corpora, models
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.preprocessing import normalize;
from sklearn import decomposition;

/Users/gimjaewon/opt/anaconda3/envs/venv/lib/python3.7/site-packages/botocore/vendored/requests/packages/urllib3/_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping, MutableMapping
/Users/gimjaewon/opt/anaconda3/envs/venv/lib/python3.7/site-packages/botocore/vendored/requests/packages/urllib3/_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping, MutableMapping
/Users/gimjaewon/opt/anaconda3/envs/venv/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
/Users/gimjaewon/opt/anaconda3/envs/venv/l

In [2]:
# 데이터 읽어오기 - 특정 기간에 한해서

df = pd.read_csv('adoor_data/answers.csv', encoding='UTF8')
df['created_at'] = pd.to_datetime(df['created_at'])

start_date = pd.Timestamp(2019, 1, 28, 0)
end_date = pd.Timestamp(2019, 4, 1, 0)

mask = (df['created_at'] > start_date) & (df['created_at'] <= end_date)
df = df.loc[mask]

df.head()

,id,author_id,question_id,content,tag_string,created_at,updated_at
0,1,5,236,"오늘 상담쌤과 칭찬에 대한 대화를 나눴다. 내가 애들의 칭찬을 잘 못 믿는, 그리고...",NaN,2019-01-28 14:11:42.543635,2019-01-28 14:11:42.543635
1,2,5,103,원하는 순간에 생각의 흐름을 멈추고 하고자 하는 일에 집중할 수 있는 능력.,NaN,2019-01-28 14:12:09.623446,2019-01-28 14:12:09.623446
2,3,5,98,가만히 공상/유튜브. 혼자 있는 시간.,NaN,2019-01-28 14:13:16.547800,2019-01-28 14:13:16.547800
3,4,5,95,게으름. 그 중에서도 요즘 가장 싫은건 게으름으로 인해 더러워진 방. 너무 짜증나서...,NaN,2019-01-28 14:14:38.838070,2019-01-28 14:14:38.838070
4,6,4,5,진수니랑 같이 프론트 더 멋있게 만들기!!!!!\r\n,NaN,2019-01-28 15:55:14.088330,2019-01-28 15:55:14.088330


In [3]:
# 형태소 분석

twitter = Okt()
pos = lambda d: ['/'.join(p) for p in twitter.pos(d, stem=True, norm=True)]

texts_ko = []
content = df.content
docs_ko = ""

for row in content:
    text = row
    if not pd.isna(text):
        docs_ko = docs_ko + text
        for morph in pos(text):
            if (morph.split('/')[1]) in ['Noun'] and len(morph.split('/')[0]) > 1:
                m = []
                m.append(morph)
                texts_ko.append(m)

In [4]:
dictionary_ko = corpora.Dictionary(texts_ko)
dictionary_ko.save('ko.dict')

In [5]:
# tfidf 변환

tf_ko = [dictionary_ko.doc2bow(text) for text in texts_ko]
tfidf_model_ko = models.TfidfModel(tf_ko)
tfidf_ko = tfidf_model_ko[tf_ko]
corpora.MmCorpus.serialize('ko.mm', tfidf_ko)

In [33]:
# LDA 모델 - tfidf 변환을 하지 않은 경우

ntopics, nwords = 5, 7

lda_ko = models.ldamodel.LdaModel(tf_ko, id2word=dictionary_ko, num_topics=ntopics)
lda_ko.print_topics(num_words=nwords)

[(0,
  '0.127*"생각/Noun" + 0.028*"때문/Noun" + 0.027*"오늘/Noun" + 0.025*"하나/Noun" + 0.025*"인생/Noun" + 0.018*"자신/Noun" + 0.016*"정도/Noun"'),
 (1,
  '0.020*"친구/Noun" + 0.019*"가장/Noun" + 0.018*"손흥민/Noun" + 0.015*"사랑/Noun" + 0.013*"기분/Noun" + 0.012*"시간/Noun" + 0.010*"조금/Noun"'),
 (2,
  '0.142*"사람/Noun" + 0.037*"마음/Noun" + 0.017*"다른/Noun" + 0.017*"질문/Noun" + 0.014*"보고/Noun" + 0.014*"상처/Noun" + 0.012*"이야기/Noun"'),
 (3,
  '0.031*"진짜/Noun" + 0.020*"그냥/Noun" + 0.018*"사실/Noun" + 0.014*"욕망/Noun" + 0.012*"의지/Noun" + 0.010*"노래/Noun" + 0.010*"수도/Noun"'),
 (4,
  '0.031*"기억/Noun" + 0.029*"정말/Noun" + 0.025*"지금/Noun" + 0.018*"요즘/Noun" + 0.017*"항상/Noun" + 0.015*"감정/Noun" + 0.014*"첫사랑/Noun"')]

In [7]:
# LDA 모델 - tfidf 변환을 적용한 경우

lda_ko = models.ldamodel.LdaModel(tfidf_ko, id2word=dictionary_ko, num_topics=ntopics)
lda_ko.print_topics()

[(0,
  '0.065*"사람/Noun" + 0.017*"진짜/Noun" + 0.017*"기억/Noun" + 0.016*"정말/Noun" + 0.014*"지금/Noun" + 0.013*"때문/Noun" + 0.012*"손흥민/Noun" + 0.010*"요즘/Noun" + 0.008*"자신/Noun" + 0.008*"정도/Noun"'),
 (1,
  '0.021*"마음/Noun" + 0.015*"가장/Noun" + 0.011*"항상/Noun" + 0.010*"감정/Noun" + 0.010*"욕망/Noun" + 0.008*"의지/Noun" + 0.008*"얘기/Noun" + 0.007*"세상/Noun" + 0.007*"어제/Noun" + 0.007*"의미/Noun"'),
 (2,
  '0.067*"생각/Noun" + 0.015*"오늘/Noun" + 0.014*"친구/Noun" + 0.013*"하나/Noun" + 0.013*"인생/Noun" + 0.012*"그냥/Noun" + 0.011*"사실/Noun" + 0.010*"사랑/Noun" + 0.009*"기분/Noun" + 0.009*"다른/Noun"')]

In [8]:
ldatopics = lda_ko.show_topics(formatted=False)
ldatopics[0]

(0,
 [('사람/Noun', 0.064835265),
  ('진짜/Noun', 0.01744445),
  ('기억/Noun', 0.016596146),
  ('정말/Noun', 0.016434476),
  ('지금/Noun', 0.013660572),
  ('때문/Noun', 0.013271252),
  ('손흥민/Noun', 0.011655125),
  ('요즘/Noun', 0.010084574),
  ('자신/Noun', 0.008331152),
  ('정도/Noun', 0.007950612)])

In [9]:
# LSI (Latent Semantic Indexing) 모델

lsi_ko = models.lsimodel.LsiModel(tfidf_ko, id2word=dictionary_ko, num_topics=ntopics)
lsi_ko.print_topics()

[(0,
  '-1.000*"사람/Noun" + -0.000*"대화/Noun" + -0.000*"느낌/Noun" + -0.000*"누구/Noun" + 0.000*"점점/Noun" + 0.000*"알바/Noun" + -0.000*"최고/Noun" + 0.000*"먼저/Noun" + -0.000*"상담/Noun" + -0.000*"음악/Noun"'),
 (1,
  '-1.000*"생각/Noun" + 0.000*"누구/Noun" + 0.000*"보기/Noun" + -0.000*"상담/Noun" + -0.000*"노력/Noun" + -0.000*"편이/Noun" + -0.000*"최고/Noun" + -0.000*"아마/Noun" + -0.000*"학교/Noun" + 0.000*"최근/Noun"'),
 (2,
  '1.000*"정말/Noun" + 0.005*"엄마/Noun" + 0.005*"자기/Noun" + -0.004*"표현/Noun" + 0.004*"아주/Noun" + -0.004*"관계/Noun" + -0.004*"동안/Noun" + -0.004*"정리/Noun" + -0.004*"연락/Noun" + 0.004*"상담/Noun"')]

In [10]:
# HDP (Hierarchical Dirichlet Process) 모델 - n_topics printed ONLY

hdp_ko = models.hdpmodel.HdpModel(tfidf_ko, id2word=dictionary_ko)
hdp_ko.print_topics(ntopics, nwords)

/Users/gimjaewon/opt/anaconda3/envs/venv/lib/python3.7/site-packages/gensim/models/hdpmodel.py:467: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  start_time = time.clock()


[(0,
  '0.003*바디/Noun + 0.003*연주/Noun + 0.003*신고/Noun + 0.003*등등/Noun + 0.002*질문/Noun'),
 (1,
  '0.003*아마/Noun + 0.003*어쩜/Noun + 0.002*하하/Noun + 0.002*라벨/Noun + 0.002*자주/Noun'),
 (2,
  '0.003*해지/Noun + 0.003*도태/Noun + 0.003*햇살/Noun + 0.003*먼저/Noun + 0.003*안좋듀/Noun')]

In [11]:
# Bag of Words

bow = tfidf_model_ko[dictionary_ko.doc2bow(texts_ko[0])]
sorted(lda_ko[bow], key=lambda x: x[1], reverse=True)

[(2, 0.6663449), (0, 0.16683903), (1, 0.16681607)]

In [12]:
sorted(hdp_ko[bow], key=lambda x: x[1], reverse=True)

[(95, 0.5033288556332016)]

In [13]:
bow = tfidf_model_ko[dictionary_ko.doc2bow(texts_ko[1])]
sorted(lda_ko[bow], key=lambda x: x[1], reverse=True)

[(0, 0.6664888), (1, 0.16675824), (2, 0.1667529)]

In [14]:
sorted(hdp_ko[bow], key=lambda x: x[1], reverse=True)

[(142, 0.5032619482630247)]

In [15]:
# # Coherence 수준 측정 - LDA 모델

# def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
#     coherence_values = []
#     model_list = []
#     for num_topics in range(start, limit, step):
#         model = models.LdaModel(corpus=tf_ko, num_topics=num_topics, id2word=dictionary_ko)
#         model_list.append(model)
#         coherencemodel = models.CoherenceModel(model=model, texts=texts_ko, dictionary=dictionary_ko, coherence='c_v')
#         coherence_values.append(coherencemodel.get_coherence())

#     return model_list, coherence_values

In [16]:
# coherence_model_lda = models.CoherenceModel(model=lda_ko, texts=texts_ko, dictionary=dictionary_ko, coherence='c_v')
# coherence_lda = coherence_model_lda.get_coherence()
# print('\nCoherence Score: ', coherence_lda)

In [17]:
# model_list, coherence_values = compute_coherence_values(
#     dictionary=dictionary_ko, 
#     corpus=tf_ko, 
#     texts=texts_ko, 
#     start=1, 
#     limit=5, 
#     step=1)

In [18]:
# limit=5; start=1; step=1;
# x = range(start, limit, step)
# plt.rcParams["font.size"] = 20
# plt.figure(figsize=(9,5))
# plt.plot(x, coherence_values)
# plt.xlabel("Num Topics")
# plt.ylabel("Coherence score")
# plt.legend(("coherence_values"), loc='best')
# plt.show()

In [19]:
# for m, cv in zip(x, coherence_values):
#     print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [20]:
# optimal_model = model_list[1]
# model_topics = optimal_model.show_topics(formatted=False)
# optimal_model.print_topics(num_words=10)

In [21]:
# # Topics Sentence Formation

# def format_topics_sentences(ldamodel=lda_ko, corpus=tf_ko, texts=df['content'].values.astype('U')):
   
#     sent_topics_df = pd.DataFrame()

   
#     for i, row in enumerate(ldamodel[corpus]):
#         row = sorted(row, key=lambda x: (x[1]), reverse=True)
#         # Get the Dominant topic, Perc Contribution and Keywords for each document
#         for j, (topic_num, prop_topic) in enumerate(row):
#             if j == 0:  # -- dominant topic
#                 wp = ldamodel.show_topic(topic_num)
#                 topic_keywords = ", ".join([word for word, prop in wp])
#                 sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
#             else:
#                 break
#     sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

#     # Add original text to the end of the output
    
#     contents = pd.Series(texts)
#     sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
#     return(sent_topics_df)
    

# df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=tf_ko, texts=df['content'].values.astype('U'))


# df_dominant_topic = df_topic_sents_keywords.reset_index()
# df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']


# df_dominant_topic.head(5)

In [22]:
path='/Library/Fonts/NanumGothic.ttf'
font_name = fm.FontProperties(fname=path, size=50).get_name()
matplotlib.rc('font', family=font_name)
matplotlib.rc('axes', unicode_minus=False)
plt.rc('font', family=font_name)
plt.rc('axes', unicode_minus=False)

In [23]:
# LDA Model w/ 3 Topics

data_lda = {i: OrderedDict(lda_ko.show_topic(i, 5)) for i in range(ntopics)}
df_lda = pd.DataFrame(data_lda)
df_lda = df_lda.fillna(0).T
print(df_lda.shape)
df_lda

(3, 15)


,사람/Noun,진짜/Noun,기억/Noun,정말/Noun,지금/Noun,마음/Noun,가장/Noun,항상/Noun,감정/Noun,욕망/Noun,생각/Noun,오늘/Noun,친구/Noun,하나/Noun,인생/Noun
0,0.064835,0.017444,0.016596,0.016434,0.013661,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.020976,0.014716,0.011139,0.009737,0.009524,0.00000,0.00000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.06678,0.01471,0.013945,0.013298,0.013125


In [24]:
# 3-topic LDA 모델 - MDS Mapping

corpus_lda = lda_ko[tfidf_ko]

pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda_ko, corpus_lda, dictionary_ko, mds='tsne')
panel

PreparedData(topic_coordinates=                 x            y  topics  cluster       Freq
topic                                                      
2     -1015.700562  1338.981201       1        1  49.262043
0      -387.996002  -373.316162       2        1  33.988113
1     -2184.745605   -60.565800       3        1  16.749844, topic_info=         Term        Freq       Total Category  logprob  loglift
74    사람/Noun  163.000000  163.000000  Default  30.0000  30.0000
10    생각/Noun  240.000000  240.000000  Default  29.0000  29.0000
350   마음/Noun   26.000000   26.000000  Default  28.0000  28.0000
342   진짜/Noun   43.000000   43.000000  Default  27.0000  27.0000
277   기억/Noun   41.000000   41.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
295   사고/Noun    5.052511    5.989573   Topic3  -5.4788   1.6166
1365  의미/Noun    8.355627    9.941463   Topic3  -4.9757   1.6130
37    기도/Noun    4.677465    5.556454   Topic3  -5.5559   1.6146
16    혼자/Noun    4.618757    5.503131   Topic3  -5.5685   1.6116
1381  단어/Noun    6.462346    9.161692   Topic3  -5.2327   1.4377

[133 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
20        1  0.052803  가장/Noun
20        2  0.052803  가장/Noun
20        3  0.950452  가장/Noun
1434      1  0.947564  감성/Noun
1434      2  0.049872  감성/Noun
...     ...       ...      ...
666       2  0.958753  해도/Noun
40        1  0.956891  행복/Noun
301       3  0.974173  현실/Noun
16        1  0.181715  혼자/Noun
16        3  0.908574  혼자/Noun

[155 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2])

In [42]:
def getNVM_lemma(text):
    tokenizer = MeCab.Tagger()
    parsed = tokenizer.parse(text)
    word_tag = [w for w in parsed.split("\n")]
    pos = []
#     tags = ['NNG','NNP','VV','VA', 'VX', 'VCP','VCN']
    tags = ['NNG','NNP']
    for word_ in word_tag[:-2]:
        word = word_.split("\t")
        tag = word[1].split(",")
        if(len(word[0]) < 2) or ("게" in word[0]):
            continue
        if(tag[-1] != '*'):
            t = tag[-1].split('/')
            if(len(t[0]) > 1 and ('VV' in t[1] or 'VA' in t[1] or 'VX' in t[1])):
                pos.append(t[0])
        else:
            if(tag[0] in tags):
                pos.append(word[0])
    return pos

In [43]:
vectorizer = CountVectorizer(tokenizer=getNVM_lemma, min_df=2)
x_counts = vectorizer.fit_transform(df['content'].values.astype('U'))
print( "Created %d X %d document-term matrix" % (x_counts.shape[0], x_counts.shape[1]) )
transformer = TfidfTransformer(smooth_idf=False);
x_tfidf = transformer.fit_transform(x_counts);

Created 1005 X 747 document-term matrix


In [44]:
terms = vectorizer.get_feature_names()
print("Vocabulary has %d distinct terms" % len(terms))

Vocabulary has 747 distinct terms


In [45]:
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)
model = NMF(n_components=10, init='nndsvd');
model.fit(xtfidf_norm)

NMF(init='nndsvd', n_components=10)

In [46]:
def get_nmf_topics(model, n_top_words):
    
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(num_topics):
        
        words_ids = model.components_[i].argsort()[:-10 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [47]:
num_topics = 5
nmf_df = get_nmf_topics(model, 2)
nmf_df

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05
0,사람,생각,친구,시간,사랑
1,기대,모르,만나,자신,엄마
2,만나,행동,남자,기대,원망
3,대하,예전,대화,버스,신뢰
4,편하,열정,도어,수업,대신
5,도어,공상,노래,통화,여름
6,자신,변하,힘들,인연,재미
7,목표,짜증,고백,오늘,능력
8,이야기,평생,기분,공상,마음
9,버리,들어오,차오르,유튜브,동안


In [40]:
num_topics = 5
nmf_df = get_nmf_topics(model, 5)
nmf_df

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05
0,사람,생각,시간,친구,사랑
1,만나,모르,자신,만나,엄마
2,대하,행동,기대,남자,원망
3,편하,열정,버스,대화,신뢰
4,도어,예전,수업,도어,대신
5,자신,공상,통화,노래,재미
6,목표,변하,인연,힘들,능력
7,이야기,짜증,오늘,고백,마음
8,버리,들어오,공상,기분,여름
9,이유,평생,유튜브,차오르,동안


In [32]:
num_topics = 10
nmf_df = get_nmf_topics(model, 5)
nmf_df

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,생각,사람,모르,친구,어렵,사랑,배우,시간,행복,학교
1,행동,보이,돌아오,남자,성격,엄마,재미,일어나,인생,파이
2,괜찮,만나,대답,대화,변하,원망,가르침,버스,아름답,형태
3,예전,자신,바꾸,그렇,평생,아니,가르치,자신,하당,공연
4,들어오,힘들,만드,만나,고민,신뢰,감사,지나,건강,연극
5,평생,편하,부탁,노래,만들,갔다오,반성,기대,아니,리스트
6,정도,버리,구체,힘들,질문,대신,악기,수업,미치,소원
7,보이,이야기,도어,기쁘,택하,재미,입학,오늘,평생,알차
8,머리,목표,알아주,진심,에어컨,능력,대학,돌아보,하루,피곤
9,멋지,좋아하,따르,친하,요소,마음,여유,통화,진심,만나
